# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
* Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
* Aggregate the Yelp Data
    * Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    * Aggregate "count" features using this same logic
    * Combine the review document for a restaurant into a CLOB using the same logic
* LEFT JOIN the violations dataset to the Yelp data after aggregationis complete for both datasets

In [125]:
import psycopg2 as psy
import pandas as pd
import re

In [126]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables"
            " WHERE table_schema='public'" 
            " AND table_type='BASE TABLE'")
rows =cur.fetchall()
print(pd.DataFrame(rows))

                       0
0               business
1                checkin
2             trnt_insps
3  violations_pittsburgh
4                   tips
5                reviews


In [127]:
cur.execute("SELECT * FROM public.restaurants LIMIT 50")
biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.checkin LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

In [128]:
biz.columns = ['bizID', 'name', 'address', 'zip', 'neighborhood', 'lat','long', 'categories','attributes','is_open','review_count','hours','stars']
biz.describe()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,review_count,hours,stars
count,50,50,50,50,50,50,50,50,50,50,50,50,50
unique,50,49,50,50,32,50,50,49,49,2,31,30,8
top,qEiyiXdSRlpwPvWKB5ftdA,Tim Hortons,1736 Queen Street W,M1E,,43.6511571,-79.315406,"['Restaurants', 'Italian']",,1,3,,3.5
freq,1,2,1,1,7,1,1,2,2,39,10,20,19


In [129]:
reviews.columns = ['type','cool','bizID','reviewID','userID','stars','text','useful','funny','date']
#get dummies for star rating column
reviews = pd.concat([reviews, pd.get_dummies(reviews['stars'], prefix='stars')], axis=1)
reviews.head()

,type,cool,bizID,reviewID,userID,stars,text,useful,funny,date,stars_1,stars_2,stars_3,stars_4,stars_5
0,b'review',0,b'2aFiy99vNLklCx3T_tGS9A',b'NxL8SIC5yqOdnlXCg18IBg',b'KpkOkG6RIf4Ra25Lhhxf1A',5,"b""If you enjoy service by someone who is as co...",0,0,b'2011-10-10',0.0,0.0,0.0,0.0,1.0
1,b'review',0,b'2aFiy99vNLklCx3T_tGS9A',b'pXbbIgOXvLuTi_SPs1hQEQ',b'bQ7fQq1otn9hKX-gXRsrgA',5,"b""After being on the phone with Verizon Wirele...",1,0,b'2010-12-29',0.0,0.0,0.0,0.0,1.0
2,b'review',0,b'2aFiy99vNLklCx3T_tGS9A',b'wslW2Lu4NYylb1jEapAGsw',b'r1NUhdNmL6yU9Bn-Yx6FTw',5,b'Great service! Corey is very service oriente...,0,0,b'2011-04-29',0.0,0.0,0.0,0.0,1.0
3,b'review',1,b'2LfIuF3_sX6uwe-IR-P0jQ',b'GP6YEearUWrzPtQYSF1vVg',b'aW3ix1KNZAvoM8q-WghA3Q',5,"b""Highly recommended. Went in yesterday lookin...",0,0,b'2014-07-14',0.0,0.0,0.0,0.0,1.0
4,b'review',0,b'2LfIuF3_sX6uwe-IR-P0jQ',b'25RlYGq2s5qShi-pn3ufVA',b'YOo-Cip8HqvKp_p9nEGphw',4,"b""I walked in here looking for a specific piec...",0,0,b'2014-01-15',0.0,0.0,0.0,1.0,0.0


In [131]:
checkins.columns = ['bizID','type','datetime']
checkins.describe()

,bizID,type,datetime
count,50,50,50
unique,50,1,50
top,b'gOeU8MmVBzmaOWA191hLpg',b'checkin',"['Thu-12:1', 'Fri-15:1', 'Sat-15:2', 'Tue-15:1..."
freq,1,50,1


In [132]:
#this is to strip the 'b' tags from bizID, delete if done in SQL before import
checkins['bizID'] = checkins['bizID'].str[2:24]
reviews['bizID'] = reviews['bizID'].str[2:24]
biz['bizID'] = biz['bizID'].str[2:24]


In [137]:
#get list of unique biz and create df
uniquebiz = []
for x in reviews['bizID']:
    if x not in uniquebiz:
        uniquebiz.append(x)
bizrevs = pd.DataFrame(uniquebiz)
bizrevs.columns = ['bizID']
bizrevs = pd.merge(bizrevs,biz[['bizID','name','address']],on='bizID', how='left')

**Note: merge in cell above won't result in matches for name/address because of 50 row limit put in place. should work after removing limit**

In [138]:
#declare rest of columns
bizrevs['reviews'] = ""
bizrevs['checkins'] = 0
bizrevs['stars_1'] = 0
bizrevs['stars_2'] = 0
bizrevs['stars_3'] = 0
bizrevs['stars_4'] = 0
bizrevs['stars_5'] = 0
bizrevs['reviews_whole_words'] = ""

#look up each unique biz ID in reviews table
for i in range(len(bizrevs['bizID'])):
    #add text of review to reviews column if biz IDs match
    for x in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][x]:
            bizrevs['reviews'][i] = bizrevs['reviews'][i] + reviews['text'][x]
    #count number of checkins
    for y in range(len(checkins['bizID'])):
        if bizrevs['bizID'][i] == checkins['bizID'][y]:
            bizrevs['checkins'][i] = bizrevs['checkins'][i] + 1
    #count number of reviews with each star rating
    for z in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][z]:
            bizrevs['stars_1'][i] = bizrevs['stars_1'][i] + reviews['stars_1'][z]
            bizrevs['stars_2'][i] = bizrevs['stars_2'][i] + reviews['stars_2'][z]
            bizrevs['stars_3'][i] = bizrevs['stars_3'][i] + reviews['stars_3'][z]
            bizrevs['stars_4'][i] = bizrevs['stars_4'][i] + reviews['stars_4'][z]
            bizrevs['stars_5'][i] = bizrevs['stars_5'][i] + reviews['stars_5'][z]
    #extract whole words from reviews
    bizrevs['reviews_whole_words'][i] = ' '.join(re.findall('[A-Za-z]+', bizrevs['reviews'][i]))
    
bizrevs.head()

C:\Users\Arnab\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Arnab\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Arnab\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Arnab\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,reviews_whole_words
0,2aFiy99vNLklCx3T_tGS9A,NaN,NaN,"b""If you enjoy service by someone who is as co...",0,0,0,0,0,3,b If you enjoy service by someone who is as co...
1,2LfIuF3_sX6uwe-IR-P0jQ,NaN,NaN,"b""Highly recommended. Went in yesterday lookin...",0,2,0,0,4,9,b Highly recommended Went in yesterday looking...
2,4P-vTvE6cncJyUyLh73pxw,NaN,NaN,"b""Unmmmm, no. It's a bar with tables in the ne...",0,2,1,5,7,2,b Unmmmm no It s a bar with tables in the next...
3,0czfEgv9KAD4VlIa7ANPWQ,NaN,NaN,"b""Overall, I'll never go back. Rewinding to th...",0,1,3,3,1,2,b Overall I ll never go back Rewinding to the ...
4,7GI_V9oLCUGdn2ogqB0IBg,NaN,NaN,"b""I highly doubt anyone uses Yelp to find a pl...",0,1,0,0,0,2,b I highly doubt anyone uses Yelp to find a pl...


In [58]:
#import toronto inspection data
cur.execute("SELECT * FROM public.trnt_inspn_agg LIMIT 50")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['ID','name','address','date','count_minor','count_significant','count_crucial','count_na','total_count_cs']
insp.head()

,ID,name,address,date,count_minor,count_significant,count_crucial,count_na,total_count_cs
0,10348267,OLLIFFE,1097 YONGE ST,11/30/15,2,0,0,0,0
1,10378852,CORNED BEEF HOUSE,270 ADELAIDE ST W,5/13/15,1,0,0,0,0
2,10377653,BLOOR STREET MARKET,55 BLOOR ST W,1/30/17,1,2,0,0,2
3,10414022,BIRYANI KING,680 REXDALE BLVD,11/17/15,0,0,0,0,0
4,9013311,SAIGON FLOWER RESTAURANT,1138 QUEEN ST W,7/15/15,0,0,0,0,0
